In [5]:
import os
from sys import platform

import torchaudio

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_voice


ROOT = os.getcwd()

modelDir = os.path.join(ROOT, "models")
outputDir = os.path.join(ROOT, "output")
outputFiles = [file for file in os.listdir(outputDir) if ".DS_Store" not in file]
modelFiles = [file for file in (os.listdir(modelDir)) if ".DS_Store" not in file]
newModels = sorted(
    [
        (int(file.replace("_gpt.pth", "")), file)
        for file in modelFiles
        if file.replace("_gpt.pth", ".wav") not in outputFiles
    ],
    reverse=True,
)

In [6]:
len(newModels)

1

In [7]:
for model, model_file in newModels:
    model_path = f"models/{model_file}"
    autoregressive_batch_size = 8 if platform == "darwin" else None
    tts = TextToSpeech(
        model_path=model_path,
        use_deepspeed=True,
        kv_cache=True,
        autoregressive_batch_size=autoregressive_batch_size,
    )

    text = "You know, liberalism used to be something my dad would get into fights over in bars."

    preset = "fast"

    voice = "john"

    voice_samples, conditioning_latents = load_voice(voice)
    gen = tts.tts_with_preset(
        text,
        voice_samples=voice_samples,
        conditioning_latents=conditioning_latents,
        preset=preset,
    )
    torchaudio.save(f"output/{model}.wav", gen.squeeze(0).cpu(), 24000)
    print(f"{model} saved")
    del tts

Some weights of the model checkpoint at jbetker/wav2vec2-large-robust-ft-libritts-voxpopuli were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_v', 'wav2vec2.encoder.pos_conv_embed.conv.weight_g']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at jbetker/wav2vec2-large-robust-ft-libritts-voxpopuli and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.o

Generating autoregressive samples..


100%|██████████| 12/12 [03:33<00:00, 17.80s/it]


Computing best candidates using CLVP


100%|██████████| 12/12 [00:11<00:00,  1.06it/s]


Transforming autoregressive outputs into audio..


100%|██████████| 80/80 [02:09<00:00,  1.62s/it]


53400 saved


In [8]:
import IPython.display as ipd

model, model_file = newModels[0]

ipd.Audio(f"output/{model}.wav")

In [ ]:
import librosa

durations = sorted(
    [(librosa.get_duration(path=f"output/{file}"), file) for file in outputFiles]
)


selected = sorted(
    [file for duration, file in durations if duration < 7],
    key=lambda x: int(x.replace(".wav", "")),
)